In [1]:
p = 0xC2B0C8D9BFE064019955C4A3819E31BFA6679B32BC5A2021D1EDC5EA83BCCFD79946200A6D5C7D349B69B1BE3EE01AF3C48811D9B95BD8740BB5EA5B3BD1CAB5
q = 0xFC932034EA8B970B000C6C1CBBB7ACAB0ECC6D7C86840B75C9A2EAFC84536D04DF4F63B49B7852D65CC9FA313EB2E095C5A1B3E47BF9550447E65907741A43F9

In [2]:
import os
import base64
import gensafeprime

def primegen(n=2048):
    """Thats a reallly big prime generated using OpenSSL. It will do."""
    #return number.getPrime(n, os.urandom

def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def invmod(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

assert (invmod(15, 37) * 15) % 37 == 1
assert invmod(17, 3120) == 2753

def str_to_int(s):
    return int(base64.b16encode(s), 16)
def int_to_str(i):
    s = hex(i)[2:].upper()
    if len(s) % 2 == 1:
        s = '0' + s # base16 wants even length strings, adding a 0 in front does not change the value. Perfect.
    return base64.b16decode(s)
assert int_to_str(str_to_int((b"Hello, world !"))) == b'Hello, world !'

class RSA:
    def __init__(self, p=None, q=None):
        self.p = p or primegen()
        self.q = q or primegen()
        self.n = self.p * self.q
        self.et = (self.p - 1) * (self.q - 1)
        self.e = 3
        self.d = invmod(self.e, self.et)
    
    @property
    def public(self):
        return self.e, self.n
    
    @property
    def private(self):
        return self.d, self.n
    
    def encrypt(self, s, i2s = True):
        if isinstance(s, bytes):
            m = str_to_int(s)
            if m ** 3 < self.n:
                print('Unsafe encryption.') # If the cubed message does not wrap around, it's not much use...
            if i2s:
                return int_to_str(pow(m, self.e, self.n))
            else:
                return pow(m, self.e, self.n)
        else:
            m = s
            return pow(m, self.e, self.n)
    
    def decrypt(self, s, i2s = True):
        if isinstance(s, bytes):
            c = str_to_int(s)
            if i2s:
                return int_to_str(pow(c, self.d, self.n))
            else:
                return pow(c, self.d, self.n)
        else:
            c = s
            return pow(c, self.d, self.n)
        
    
r = RSA(p=p, q=q)
assert r.decrypt(r.encrypt(b"\xFF" * 50 + b"Vanilla Ice Baby")) == b"\xFF" * 50 + b'Vanilla Ice Baby'

In [12]:
import os
import base64
import random

B = 2**240
def oracle(encrypter, cypher):
    return 2*B <= encrypter.decrypt(cypher, i2s=False) < 3*B - 1

def non_null_random_byte():
    b = os.urandom(1)[0]
    if b != 0:
        return hex(b)[2:].zfill(2)
    else:
        return non_null_random_byte()

def pkcs_pad(message):
    """Pad a message in PKCS#1.5 fashion"""
    padded_message = "00" + hex(str_to_int(message))[2:]
    if len(padded_message) % 2 == 1:
        padded_message = "0" + padded_message # Ensure byte alignement
    while len(padded_message) + len("0002") < 64: # we use 256 bit messages
        padded_message = non_null_random_byte() + padded_message
    padded_message = "0002" + padded_message
    return int(padded_message, base=16)

def pkcs_unpad(message):
    pass

message = b"Kick it, CC"
pkcs_message = pkcs(message)
cypher = r.encrypt(pkcs_message, i2s=False)
assert oracle(r, cypher)

0002e8de9447414414b52b515345afb60a40cb44004b69636b2069742c204343


In [225]:
import tqdm
lo, hi, c = 0, r.n, cypher
cypher_2 = r.encrypt(2, i2s=False)
pbar = tqdm.tqdm(range(int(math.log(r.n, 2))+20))
for i in pbar:
    pbar.set_description(str(int_to_str(hi)))
    if oracle(r, c*cypher_2):
        lo, hi, c = (hi+lo)//2, hi, c*cypher_2 - r.n
    else:
        lo, hi, c = lo, (hi+lo)//2, c*cypher_2

b"That's why I found you don't play around with the Funky Cold Medin\x1a": 100%|██████████| 1043/1043 [00:35<00:00, 29.36it/s]                                                                                                                                                                                                                                                                                                                     


In [223]:
int_to_str(hi)

b"That's why I found you don't play around with the Funky Cold Medin\x1b"

In [188]:
hi-lo

0

In [3]:
len(hex(p*q))-2

256